In [ ]:
## https://huggingface.co/hfl/llama-3-chinese-8b

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="hfl/llama-3-chinese-8b", cache_dir = "./")

/Users/minkexiu/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /hfl/llama-3-chinese-8b/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x28d940510>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/hfl/llama-3-chinese-8b/resolve/main/config.json


'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /hfl/llama-3-chinese-8b/resolve/main/model-00001-of-00004.safetensors (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1767be5d0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/hfl/llama-3-chinese-8b/resolve/main/model-00001-of-00004.safetensors


ValueError: Could not load model hfl/llama-3-chinese-8b with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.llama.modeling_llama.LlamaForCausalLM'>).